In [7]:
from support.datatools import *
from support.paths import *
from objectives.logist import *
%pylab inline
%load_ext autoreload
%autoreload 1
%aimport support.datatools

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


`%matplotlib` prevents importing * from pylab and numpy


In [3]:
TEST_FOLD = str(2)
VALID_FOLD = str(3)
SHIFT = 2
BATCH_SIZE = 32
NB_EPOCH = 20
OVERSAMPLING = .7
UNDERSAMPLING = 8
LOWER = -1000
UPPER = 400
IN_SHAPE = (1, 22, 22)
CPU = 6

In [8]:
# train, valid, test = extract_paths(VALID_FOLD, TEST_FOLD)
train_generator = batch_generator(train,
                                  batch_size=BATCH_SIZE, 
                                  in_shape=IN_SHAPE,
                                  lower=LOWER,
                                  upper=UPPER,
                                  shift=SHIFT,
                                  undersampling=UNDERSAMPLING,
                                  oversampling=OVERSAMPLING,
                                  CPU=CPU)

valid_generator = batch_generator(valid,
                                  batch_size=BATCH_SIZE, 
                                  in_shape=IN_SHAPE,
                                  lower=LOWER,
                                  upper=UPPER,
                                  shift=0, 
                                  undersampling=0,
                                  oversampling=1,
                                  CPU=CPU)

In [9]:
class LR:
    def __init__(self, lr=1, dimensions=484,
                 l1=1e-4, l2=1e-4, 
                 num_iter=1000, verbose=0):
        
        self.l1 = l1
        self.l2 = l2
        self.lr = lr
        self.verbose = verbose
        self.num_iter = num_iter
        self.w = np.random.randn(dimensions)
        self.w = clip(self.w, -3, 3)

    def fit_generator(self, train_generator, 
                      iterations, nb_epoch, 
#                       validation_data, 
#                       nb_val_samples
                     ):
        """
        Обучение логистической регрессии.
        Настраивает self.w коэффициенты модели.
        Если self.verbose == True, то выводите значение 
        функции потерь на итерациях метода оптимизации. 
        :param X: numpy.array размера  (N, M), dtype = np.float
        :param y: numpy.array размера  (N,), dtype = np.int
        :return: self
        """
        for epoch in range(nb_epoch):
            for i in range(iterations):
                X, y = next(train_generator)
                X = X.reshape(X.shape[0], -1)
                n, d = X.shape
                w_new = self.w - self.lr * gradf(self.w, X, y, self.l1, self.l2)
                if self.verbose != 0:
                    print("Iteration " + str(i) + ". Current loss is " + str(lossf(self.w, X, y, self.l1, self.l2)))
                self.w = w_new
        return self

    def predict_proba(self, X):
        """
        Предсказание вероятности принадлежности объекта к классу 1.
        Возвращает np.array размера (N,) чисел в отрезке от 0 до 1.
        :param X: numpy.array размера  (N, M), dtype = np.float
        :return: numpy.array размера  (N,), dtype = np.int
        """
        return special.expit(np.matmul(X, self.w))

    def predict(self, X):
        """
        Предсказание класса для объекта.
        Возвращает np.array размера (N,) элементов 1 или -1.
        :param X: numpy.array размера  (N, M), dtype = np.float
        :return:  numpy.array размера  (N,), dtype = np.int
        """
        predicts = (self.predict_proba(X) > .5) * 2 - 1
        return predicts 

In [12]:
clf = LR(verbose=1)

In [13]:
clf.fit_generator(train_generator, 3, 2)

Iteration 0. Current loss is 0.0909766743763
Iteration 1. Current loss is 0.0909455189351
Iteration 2. Current loss is 0.120848960841
Iteration 0. Current loss is 0.0909611912642
Iteration 1. Current loss is 0.186210991664
Iteration 2. Current loss is 0.0919694722251
